<a href="https://colab.research.google.com/github/martinbedia/mineria/blob/main/ejercicio%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1
Sobre el código realizado durante las clases para analizar tickets estudia si los tickets
contienen parking o no. Aquellos que lo contienen aportan información adicional. Modifica el programa para tener esto en cuenta. El resultado del código deberán ser dos archivos csv. Uno tendrá las columnas id ticket, producto, cantidad, precio por unidad. El otro tendrá para cada id de ticket, parking (booleana), fecha, total precio ticket, hora entrada, hora salida, tiempo total (que podéis calcularlo de las anteriores). Las columnas para las que no se dispone de datos en algunos de los tickets deberán ser NA.

In [ ]:
pip install pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report